In [4]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import pandas as pd
import random
from random import shuffle

import time 
import datetime

In [3]:
class RobotAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.has_box = False
    
    def move(self):
        possible_moves = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = self.random.choice(possible_moves)
        self.model.grid.move_agent(self, new_position)
    
    def pickup_box(self):
        x, y = self.pos
        contents = self.model.grid.get_cell_list_contents([(x, y)])

        for content in contents:
            if content == "box" and not self.has_box:
                self.has_box = True
                return True
        return False
    
    def drop_box(self):
        x, y = self.pos
        contents = self.model.grid.get_cell_list_contents([(x, y)])

        for content in contents:
            if content == "box" and self.has_box:
                self.has_box = False
                return True
        return False

In [ ]:
class OrderBoxesModel(Model):
    def __init__(self, width, height, num_boxes, num_robots, seed):
        random.seed(seed)
        self.num_robots = num_robots
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

         # Create random stacks of boxes
        for _ in range(num_boxes):
            x, y = self.random.randrange(width), self.random.randrange(height)
            num_boxes_in_stack = self.random.randint(1, 4)
            for j in range(num_boxes_in_stack):
                self.grid.place_agent("box", (x, y))

        # Create robots
        for i in range(num_robots):
            x, y = self.random_empty_position()
            robot = RobotAgent(f"robot_{i}", self)
            self.grid.place_agent(robot, (x, y))
            self.schedule.add(robot)



        self.datacollector = DataCollector(
            agent_reporters={"Has Box": "has_box"}
        )

    def random_empty_position(self):
    # Get a random empty position in the grid
        while True:
            x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
            cell_contents = self.grid.get_cell_list_contents((x, y))
            if len(cell_contents) == 0 and all(not isinstance(agent, RobotAgent) for agent in cell_contents):
                return x, y

     def stack_boxes(self):
        stack_limit = 40  # Número máximo de pilas que queremos alcanzar
        stacks = 0  # Contador de pilas creadas

        # Recorremos todas las celdas en la cuadrícula
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                contents = self.grid.get_cell_list_contents([(x, y)])
                robots = [agent for agent in contents if isinstance(agent, RobotAgent)]
                
                # Si hay al menos un robot en la celda
                if robots:
                    robot = robots[0]  # Suponiendo que solo hay un robot por celda
                    if len(contents) >= 5 and not robot.has_box:
                        # Obtener las cajas disponibles para que el robot las recoja
                        available_boxes = [agent for agent in contents if agent == "box" and not robot.has_box]

                        # Elegir aleatoriamente si cargar o apilar las cajas
                        if available_boxes:
                            shuffle(available_boxes)
                            box_to_pick = available_boxes[0]

                            # El robot carga la caja
                            robot.pickup_box()

                            # Verificar si se ha creado una pila
                            contents_after_pickup = self.grid.get_cell_list_contents([(x, y)])
                            if len(contents_after_pickup) >= 5 and contents_after_pickup.count("box") == 5:
                                stacks += 1

                                # Detener la simulación si hemos alcanzado el límite de pilas
                                if stacks >= stack_limit:
                                    return

                            # El robot coloca la caja en otra posición aleatoria
                            new_x, new_y = self.random_empty_position()
                            robot.put_down_box()
                            box_to_pick.move_to((new_x, new_y))

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        self.stack_boxes()